# Computing Bounds on Probabilistic Safety with DeepBayes

Probabilistic Safety for BNNs: $Prob_{\theta \sim p(\theta | \mathcal{D})}\big( f^{\theta}(x') \in S \quad \forall x' \in T \big)$

In this notebook, we go over how to compute upper and lower bounds on probabilistic safety for BNNs with DeepBayes.

#### Example notebook takes 3 minutes to run in total
(Times are reported for an M1 Pro Macbook)

In [1]:
import os
import sys
import logging
import deepbayes
from deepbayes import PosteriorModel
from deepbayes.analyzers import prob_veri
from deepbayes.analyzers import FGSM

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#### Load in MNIST Dataset

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float64").reshape(-1, 28*28)
X_test = X_test.astype("float64").reshape(-1, 28* 28)

#### Define safe and unsafe predicates

These functions will take in the input upper and lower bounds as well as the values of the output 
logits and then will need to return True if the output is within the safe region i.e., $f^{\theta}(x') \in S$

In [3]:
def predicate_safe(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    if(np.argmax(worst_case) == TRUE_VALUE):
        return True
    else:
        return False
    
def predicate_unsafe(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    #worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    best_case = tf.math.add(tf.math.multiply(v1, ou), tf.math.multiply(v2, ol))
    if(np.argmax(best_case) == TRUE_VALUE):
        return False
    else:
        return True

#### Load in the pretrained BNN model

In [4]:
bayes_model = PosteriorModel("PosteriorModels/VOGN_MNIST_Posterior/")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            100480    
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


In [5]:
INDEX = 0
img = np.asarray([X_test[INDEX]])
TRUE_VALUE = np.argmax(bayes_model.predict(np.asarray([img]))) #y_test[INDEX]

#### Set Verification Parameters and compute Decision Upper Bound

##### Parameters: 
* Margin - The number of standard deviations that each weight sample will span
* Samples - The number of samples taken from the posterior (Small here for time savings)
* Max Depth - The depth of the Bonferroni Bound used to compute the probability
* Epsilon - The size of the input set that we consider for verification

In [6]:
MARGIN = 3.5
SAMPLES = 3
MAXDEPTH = 3
EPSILON = 0.01
img = np.asarray([X_test[INDEX]])
img_upper = np.clip(np.asarray([X_test[INDEX]+(EPSILON)]), 0, 1)
img_lower = np.clip(np.asarray([X_test[INDEX]-(EPSILON)]), 0, 1)
p_lower = prob_veri(bayes_model, img_lower, img_upper, MARGIN, SAMPLES, predicate=predicate_safe, depth=MAXDEPTH)
print("Lowerbound on Safety Probability: ", p_lower)

Checking Samples: 100%|██████████| 3/3 [00:00<00:00, 118.65it/s]


Found 3 safe intervals
About to compute intersection for this many intervals:  3


Computing intersection weights:   0%|          | 0/3 [00:00<?, ?it/s]/Users/matthewwicker/AdversarialRobustnessOfBNNs/deepbayes/analyzers/probverification.py:429: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  stage1_args.append((model.posterior_mean, model.posterior_var, np.swapaxes(np.asarray([weight_intervals[wi]]),1,0), margin, verbose, n_proc, False))
Computing intersection weights: 100%|██████████| 3/3 [00:00<00:00, 2405.91it/s]

Depth 1 has 3 intersections



100%|██████████| 3/3 [00:25<00:00,  8.57s/it]


Depth 1 prob:  2.813171409891387
Depth 2 has 3 intersections


100%|██████████| 3/3 [00:30<00:00, 10.32s/it]


Depth 2 prob: -2.63833808118673
Current approximation: 0.1748333287046573
Depth 2 prob::  0.1748333287046573
Depth 3 has 1 intersections


100%|██████████| 1/1 [00:27<00:00, 27.86s/it]


Depth 3 prob: 0.8249021942608562
Current approximation: 0.9997355229655135
Got this approximation:  0.9997355229655135
Lowerbound on Safety Probability:  0.9997355229655135


In [7]:
from deepbayes.analyzers import prob_veri_upper
INDEX = 1
EPSILON = 0.15
MARGIN = 3.25
img = np.asarray([X_test[INDEX]])
img_upper = np.clip(np.asarray([X_test[INDEX]+(EPSILON)]), 0, 1)
img_lower = np.clip(np.asarray([X_test[INDEX]-(EPSILON)]), 0, 1)
p_upper = prob_veri_upper(bayes_model, img_lower, img_upper, MARGIN, SAMPLES, predicate=predicate_unsafe, depth=MAXDEPTH)
p_upper = 1-p_upper
print("Upper Bound on Safety Probability: ", p_upper)

Checking Samples: 100%|██████████| 3/3 [00:12<00:00,  4.20s/it]


Found 3 safe intervals
About to compute intersection for this many intervals:  3


Computing intersection weights: 100%|██████████| 3/3 [00:00<00:00, 44150.57it/s]

Depth 1 has 3 intersections



100%|██████████| 3/3 [00:25<00:00,  8.49s/it]


Depth 1 prob:  1.9492321128510364
Depth 2 has 3 intersections


100%|██████████| 3/3 [00:30<00:00, 10.20s/it]


Depth 2 prob: -1.269194338157607
Current approximation: 0.6800377746934294
Depth 2 prob::  0.6800377746934294
Depth 3 has 1 intersections


100%|██████████| 1/1 [00:28<00:00, 28.07s/it]


Depth 3 prob: 0.2760463352902422
Current approximation: 0.9560841099836717
Got this approximation:  0.9560841099836717
Upper Bound on Safety Probability:  0.04391589001632834
